In [3]:
import pandas as pd

# Load the dataset from Excel file
diabetic_data = pd.read_excel("/Users/abdullahfiras/Desktop/data-01.xls.xls")

# Display the first few rows of the dataset
print(diabetic_data.head())


         Date   Time  Code Value  Insulin
0  04-21-1991   9:09          34       13
1  04-21-1991  17:08          62      119
2  04-21-1991  17:08          33        7
3  04-21-1991  22:51          48      123
4  04-22-1991   7:35          58      216


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load the dataset
data_path = '/Users/abdullahfiras/Desktop/data-01.xls'  # Update this path to where your dataset is located
data = pd.read_excel(data_path)

# Preprocessing
data['Datetime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'])
data.drop(['Date', 'Time'], axis=1, inplace=True)

# Categorizing 'Code Value'
code_mappings = {
    'Insulin Dose': [33, 34, 35],
    'Blood Glucose Measurement': [48, 57, 58, 59, 60, 61, 62, 63, 64],
    'Meal Ingestion': [66, 67, 68],
    'Exercise Activity': [69, 70, 71],
    'Special Event': [72],
    'Other': [65]
}
code_to_category = {code: category for category, codes in code_mappings.items() for code in codes}
data['Activity Category'] = data['Code Value'].map(code_to_category)

# Encoding 'Activity Category'
encoder = OneHotEncoder(sparse_output=False)
category_encoded = encoder.fit_transform(data[['Activity Category']])
category_df = pd.DataFrame(category_encoded, columns=encoder.get_feature_names_out(['Activity Category']))
data_preprocessed = pd.concat([data.drop(['Code Value', 'Activity Category'], axis=1), category_df], axis=1)

# Feature selection
X = data_preprocessed.drop(['Insulin', 'Datetime'], axis=1)  # Excluding 'Datetime' for simplicity
y = data_preprocessed['Insulin']

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
model = LinearRegression()
model.fit(X_train, y_train)

# Predicting and evaluating
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')


Mean Squared Error: 1968.8485863095239
Mean Absolute Error: 26.261904761904763
